# Учим w2v на сказках

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'

In [2]:
import requests
resp = requests.get(url)
text = resp.text

In [3]:
text[:300]

'ПРЕСТУПЛЕНИЕ И НАКАЗАНИЕ\nРОМАН \nВ ШЕСТИ ЧАСТЯХ С ЭПИЛОГОМ\n\nЧАСТЬ ПЕРВАЯ\n\nI\n\nВ начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С - м переулке, на улицу и медленно, как бы в нерешимости, отправился к К - ну мосту.\nОн благопо'

In [4]:
len(text)

1076543

In [5]:
text = text.lower()
print(text[:300])

преступление и наказание
роман 
в шести частях с эпилогом

часть первая

i

в начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в с - м переулке, на улицу и медленно, как бы в нерешимости, отправился к к - ну мосту.
он благопо


`!pip install nltk`

In [ ]:
import nltk 
nltk.download('punkt')

In [15]:
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text)
sentences[100]

'- я вам проценты еще за месяц внесу; потерпите.'

In [16]:
len(sentences)

13702

In [20]:
from nltk.tokenize import word_tokenize
words0 = word_tokenize(sentences[0])
words0[:5]

['преступление', 'и', 'наказание', 'роман', 'в']

In [22]:
sent_words = [word_tokenize(sent) for sent in sentences]

In [24]:
words = [word  for sent in sent_words for word in sent]
len(words)

221280

In [26]:
from collections import Counter
cnt = Counter(words)

In [32]:
' ' in words

False

In [29]:
cnt.most_common()[:10]

[(',', 25337),
 ('и', 8451),
 ('.', 8222),
 ('-', 6065),
 ('не', 3756),
 ('в', 3727),
 ('!', 3265),
 ('что', 3196),
 ('он', 2849),
 ('на', 2402)]

In [25]:
len(set(words))

26116

Стоп слова в топку!

In [ ]:
nltk.download("stopwords")

In [34]:
from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [40]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
sent_words = [[word for word in sent 
               if word not in stopwords_ru and len(word) > 1 
               and word not in string.punctuation] 
              for sent in sent_words]

In [42]:
sent_words[0]

['преступление',
 'наказание',
 'роман',
 'шести',
 'частях',
 'эпилогом',
 'часть',
 'первая',
 'начале',
 'июля',
 'чрезвычайно',
 'жаркое',
 'время',
 'вечер',
 'молодой',
 'человек',
 'вышел',
 'своей',
 'каморки',
 'которую',
 'нанимал',
 'жильцов',
 'переулке',
 'улицу',
 'медленно',
 'нерешимости',
 'отправился',
 'мосту']

In [48]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
m.normal_forms('стали')

['стать', 'сталь']

In [53]:
m.normal_forms('бутявковедами')

['бутявковед']

In [52]:
m.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [61]:
text = "Филипп пошёл в Авиньон и пленил пап!"
tokens = word_tokenize(text)

" ".join(m.normal_forms(token)[0] for token in tokens)

'филипп пойти в авиньон и пленить папа !'

In [62]:
sent_words = [[m.normal_forms(word)[0]  for word in sent 
               if word not in stopwords_ru and len(word) > 1 
               and word not in string.punctuation] 
              for sent in sent_words]

In [6]:
import re
word = 'к123о231рова132112'
re.sub('[1-9]', '', word)

'корова'

In [8]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

## Обучение w2v 

In [16]:
import string
import re

from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian') 

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

def prepare_text(text):
    tokens = word_tokenize(text.lower())
    clean_tokens = [m.normal_forms(word)[0]  for word in tokens if (word not in stopwords_ru) and 
                         (len(re.sub('[' + string.punctuation + ']', '', word)) > 1)]
    return clean_tokens

In [17]:
import requests
resp = requests.get(url)
text = resp.text

In [18]:
text_sent = sent_tokenize(text)
text_tokens =  [prepare_text(text) for text in text_sent]

In [20]:
text_tokens[101]

['добрый', 'воля', 'батюшка', 'терпеть', 'вещь', 'ваш', 'продать']

`pip install gensim`

In [31]:
from gensim.models.word2vec import Word2Vec

# size - размер вектора для каждого слова 
# window - на сколько отступать влево и вправо от целевого слова (контекст)
# min_count - минимальное число встречь со словом в тексте

model = Word2Vec(size=100, window=2, min_count=3, workers=-1)

# строим словарик из уникальных слов 
model.build_vocab(text_tokens)

# первый аргумент - выборка 
# второй аргумент - число примеров в выборке 
# третьй аргумент - сколько эпох надо учиться
model.train(text_tokens, total_examples=model.corpus_count, epochs=100)

(0, 0)

In [29]:
len(text_tokens)

13702

In [28]:
model.corpus_count

13702

In [32]:
len(model.wv.vocab)

4885

In [37]:
'процентщица' in model.wv.vocab

True

In [34]:
'ядро' in model.wv.vocab

False

In [35]:
model.wv.get_vector('старуха')[:10]

array([-0.0026579 , -0.00154479,  0.00124414,  0.00121966, -0.0004963 ,
       -0.0049302 ,  0.00495984,  0.00283323, -0.000514  , -0.00330537],
      dtype=float32)

In [36]:
model.wv.get_vector('старуха').shape

(100,)

In [38]:
model.wv.similarity('старуха', 'процентщица')

0.07684017

In [40]:
model.wv.similarity('старуха', 'старуха')

1.0000001

In [42]:
model.wv.similarity('старуха', 'топор')

0.04523443

In [44]:
model.wv.most_similar('топор')

[('оттого', 0.3332275152206421),
 ('слово', 0.3084264099597931),
 ('приговорить', 0.3083723783493042),
 ('нагибаться', 0.3036242127418518),
 ('пиво', 0.3026915490627289),
 ('раскаиваться', 0.2985680103302002),
 ('выражение', 0.2966565489768982),
 ('лом', 0.29657119512557983),
 ('облегчать', 0.2942049503326416),
 ('наливать', 0.2816503047943115)]

In [45]:
# арифметика 
# топор - раскольников + старуха 
#
# ??? -> старуха
# топор -> раскольников

model.most_similar(positive=['старуха', 'топор'], negative=['раскольников'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


[('право', 0.3476792573928833),
 ('чахотка', 0.33479034900665283),
 ('недостать', 0.29207196831703186),
 ('условие', 0.2853487432003021),
 ('вина', 0.2850760221481323),
 ('осматривать', 0.28397613763809204),
 ('покойный', 0.28018614649772644),
 ('презрительно', 0.2770897150039673),
 ('клоака', 0.26977941393852234),
 ('явка', 0.2662399709224701)]

Сохранить модель

In [50]:
model_path = './modelw2v.model'
model.save(model_path)

## Дообучение w2v

In [48]:
import requests

url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp2 = requests.get(url)
text2 = resp2.text
print(text2[:300])

Александр Сергеевич Пушкин. Сказка о царе Салтане
СКАЗКА О ЦАРЕ САЛТАНЕ,
О СЫНЕ ЕГО СЛАВНОМ И МОГУЧЕМ БОГАТЫРЕ
КНЯЗЕ ГВИДОНЕ САЛТАНОВИЧЕ И
О ПРЕКРАСНОЙ ЦАРЕВНЕ ЛЕБЕДИ



Три девицы под окном
Пряли поздно вечерком.



"Кабы я была царица,-
Говорит одна девица,-
То на весь крещеный мир
Приготовила б я


In [47]:
text_sent2 = sent_tokenize(text2)
text_tokens2 =  [prepare_text(text) for text in text_sent2]

In [53]:
model_path = './modelw2v.model'
model_old = Word2Vec.load(model_path)

In [56]:
model_old.build_vocab(text_tokens2, update=True)
model_old.train(text_tokens2, total_examples=model_old.corpus_count, epochs=100)

(0, 0)

In [57]:
'ядро' in model_old.wv.vocab

True

In [60]:
model_old.wv.get_vector('ядро')[:10]

array([ 0.00463188,  0.00128592, -0.00222403,  0.00402643,  0.00084897,
        0.00113104, -0.00017697,  0.00193736,  0.0048032 ,  0.00398187],
      dtype=float32)